<img align="centre" src="../../Supplementary_data/Github_banner.jpg" width="100%">

# Enhanced Combined Drought Index - Soil Moisture Drought Index


## Background 

Drought is an extended period, during which, fresh water availability and accessibility for the ecosystem at a given time and place is below normal, due to unfavourable spatial and temporal distribution of rainfall, temperature, soil moisture and wind characteristics [(Balint et al., 2013)](https://doi.org/10.1016/B978-0-444-59559-1.00023-2). Severe droughts can affect large populations, leading to a long-term threat to people’s livelihoods and result in tremendous economic loss [(Enenkel et al., 2016)](https://doi.org/10.3390/rs8040340).

The **Enhanced Drought Index** is a combination of the following components: Vegetation, Precipitation, Temperature and Soil moisture.

This notebook will focus on the **Soil moisture** component, which considers soil moisture deficits and deficit persistence. The index based on soil moisture is named as the Soil Moisture Drought Index (SMDI). The Soil Moisture Drought Index (SMDI) is used to assess drought conditions based on soil moisture levels.SMDI is an essential for monitoring drought conditions, especially in areas where soil moisture directly affects agriculture and natural resources. Droughts can have serious impacts on agriculture, water resources, and ecosystems, and soil moisture is a key factor in understanding drought severity, as it influences crop growth and the availability of water in the soil.


The notebook outlines:

***
1. Loading Python and DE Africa Packages
2. Setting dask cluster
3. Loading area on interest
5. Load the WAPOR soil moisture dataset
6. Resampling of data to dekadal (10-day) intervals
7. Calculate the Soil Moisture Drought Index
8. Export the result as zip

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages

In [1]:
import json
import os

import dask
import geopandas as gpd
import numpy as np
import pandas as pd
import rioxarray
import toolz
import xarray as xr
import datacube
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.datahandling import load_ard
from dekad import group_by_dekad, bin_by_interest_period, get_dekad_no_in_year, get_no_data_mask, max_consecutive_ones
from odc.geo.geobox import GeoBox
from odc.geo.geom import Geometry
from odc.geo.xr import rasterize
from pyarrow import Table
from pyarrow.parquet import write_table

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [2]:
# Connect to the datacube
dc = datacube.Datacube(app="VegetationDroughtIndex")

### Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

To use Dask, set up the local computing cluster using the cell below.

In [3]:
create_local_dask_cluster()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/victoria@kartoza.com/proxy/8787/status,
Dashboard: /user/victoria@kartoza.com/proxy/8787/status,Workers: 1
Total threads: 4,Total memory: 26.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42421,Workers: 1
Dashboard: /user/victoria@kartoza.com/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 26.21 GiB
Comm: tcp://127.0.0.1:37749,Total threads: 4
Dashboard: /user/victoria@kartoza.com/proxy/40093/status,Memory: 26.21 GiB
Nanny: tcp://127.0.0.1:41351,


### Analysis parameters

The following cell sets important parameters for the analysis:

* `country`: The country to run the analysis over.
* `resolution`: The x and y cell resolution of the satellite data in metres (spatial resolution). We'll use 5,000 m, which is approximately equal to the default CHIRPS resolution.
* `output_crs` : The coordinate reference system that the loaded data is to be reprojected to.
* `dask_chunks`:  the size of the dask chunks, dask breaks data into manageable chunks that can be easily stored in memory, e.g. `dict(x=1000,y=1000)`
* `time_range` : Time range to load data for.
* `ip` : The interest period to use to calculate the drought indices e.g. (3, 4, 5 dekads). It defines to what extent past observations are considered. Longer IPs detect more severe drought events. For example, if the IP=3 dekads, then the drought index (say PDI) of 0.35 for dekad 2 of 2006 implies actual drought for dekad 36 of 2005, dekad 1 of 2006 and dekad 2 of 2006.
* `output_dir` :  The directory in which to store results from the analysis.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.

In [4]:
resolution = (-5000, 5000)
output_crs = "EPSG:6933"
dask_chunks = dict(x=100,y=100)
time_range = ("2014","2020")
output_dir = "results"

# Corresponding to the six-months Standardized Precipitation Evapotranspiration Index
ip = 18

# Create the outpur directory if it does not exist.
os.makedirs(output_dir, exist_ok=True)

In [5]:
#loading the countries shapefiles
african_countries_url = "https://raw.githubusercontent.com/digitalearthafrica/deafrica-sandbox-notebooks/main/Supplementary_data/Rainfall_anomaly_CHIRPS/african_countries.geojson"
african_countries = gpd.read_file(african_countries_url)
print(np.unique(african_countries.COUNTRY))

['Algeria' 'Angola' 'Benin' 'Botswana' 'Burkina Faso' 'Burundi' 'Cameroon'
 'Cape Verde' 'Central African Republic' 'Chad' 'Comoros'
 'Congo-Brazzaville' 'Cote d`Ivoire' 'Democratic Republic of Congo'
 'Djibouti' 'Egypt' 'Equatorial Guinea' 'Eritrea' 'Ethiopia' 'Gabon'
 'Gambia' 'Ghana' 'Guinea' 'Guinea-Bissau' 'Kenya' 'Lesotho' 'Liberia'
 'Libya' 'Madagascar' 'Malawi' 'Mali' 'Mauritania' 'Morocco' 'Mozambique'
 'Namibia' 'Niger' 'Nigeria' 'Rwanda' 'Sao Tome and Principe' 'Senegal'
 'Sierra Leone' 'Somalia' 'South Africa' 'Sudan' 'Swaziland' 'Tanzania'
 'Togo' 'Tunisia' 'Uganda' 'Western Sahara' 'Zambia' 'Zimbabwe']


In [6]:
# Select the country of interest by providing the name below
country = "Ethiopia"
idx = african_countries[african_countries['COUNTRY'] == country].index[0]
geopolygon = Geometry(geom=african_countries.iloc[idx].geometry, crs=african_countries.crs)

# Create the output path
output_path = os.path.join(output_dir, f"SMDI_{country}.parquet")

### Load WaPOR soil moisture data
Load the WaPOR soil moisture data using the analysis parameters set in the previous section.

In [7]:
# List all available WAPOR products.
products_df =dc.list_products()
products_df[products_df.name.str.contains("wapor")]

,name,description,license,default_crs,default_resolution
name,,,,,
wapor_soil_moisture,wapor_soil_moisture,WaPOR v3 Level 2 100m dekadal relative root zo...,CC-BY-SA-4.0,EPSG:4326,"(-0.0009765625, 0.0009765625)"


In [8]:
# List the measurements for the product
dc.list_measurements().loc["wapor_soil_moisture"]

,name,dtype,units,nodata,aliases,flags_definition,add_offset,scale_factor
measurement,,,,,,,,
relative_soil_moisture,relative_soil_moisture,int16,%,-9999.0,NaN,NaN,0.0,0.001


In [10]:
%%time
ds_sm = dc.load(
    product="wapor_soil_moisture",
    measurements=["relative_soil_moisture"],
    time=time_range,
    geopolygon=geopolygon,
    resolution=resolution,
    output_crs=output_crs,
)

# Mask no data values
ds_sm = ds_sm.where(ds_sm != -9999.0)

ds_sm

CPU times: user 10.4 s, sys: 7.49 s, total: 17.9 s
Wall time: 5min 56s


<xarray.Dataset> Size: 36MB
Dimensions:                 (time: 108, y: 291, x: 290)
Coordinates:
  * time                    (time) datetime64[ns] 864B 2018-01-05T23:59:59.50...
  * y                       (y) float64 2kB 1.882e+06 1.878e+06 ... 4.325e+05
  * x                       (x) float64 2kB 3.182e+06 3.188e+06 ... 4.628e+06
    spatial_ref             int32 4B 6933
Data variables:
    relative_soil_moisture  (time, y, x) float32 36MB 572.0 485.0 ... nan nan
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

### Mask the soil moisture data

In [11]:
# Rasterize the country geopolygon and mask the loaded  Landsat data.
country_mask = rasterize(poly=geopolygon, how=ds_sm.odc.geobox)
ds_sm = ds_sm.where(country_mask)
ds_sm

<xarray.Dataset> Size: 36MB
Dimensions:                 (time: 108, y: 291, x: 290)
Coordinates:
  * time                    (time) datetime64[ns] 864B 2018-01-05T23:59:59.50...
  * y                       (y) float64 2kB 1.882e+06 1.878e+06 ... 4.325e+05
  * x                       (x) float64 2kB 3.182e+06 3.188e+06 ... 4.628e+06
    spatial_ref             int32 4B 6933
Data variables:
    relative_soil_moisture  (time, y, x) float32 36MB nan nan nan ... nan nan
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

### Resample data

Resample the soil moisture `ds_sm` timeseries into **dekadal** (10-day) timesteps.

In [13]:
# To DataArray
da_sm_dekadal = group_by_dekad(ds_sm["relative_soil_moisture"]).mean(dim="time")
da_sm_dekadal 

<xarray.DataArray 'relative_soil_moisture' (dekad: 108, y: 291, x: 290)> Size: 36MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 864B 2018-01-01 ... 2020-12-21
Attributes:
    units:         %
    nodata:        -9999
    add_offset:    0.0
    scale_factor:  0.001
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

### Group the modified timeseries using the `ip` parameter

The `ip` parameter determines to what extent past observations are considered. Longer IPs detect more severe drought events. The default `ip` used of **18 dekads** corresponds to the 6-month Standardized Precipitation Evapotranspiration Index.

For example, calculating the Soil Moisture Drought Index for the dekad `2011-01-11` using an interest period of `ip=3` requires rainfall data from the dekad `2010-12-21`, `2011-01-01` and `2011-01-11`.


**Each interest period is labelled using its end dekad.**

In [15]:
# Group the soil moisture dekadal timeseries by the interest period.
# The dictionary keys are the end dekad for each interest period.
# The values of the dictionary are the soil moisture data that belongs to the interest period
da_sm_binned_by_interest_period = bin_by_interest_period(ds=da_sm_dekadal, ip=ip)

In [16]:
# Get the interest periods
interest_periods = list(da_sm_binned_by_interest_period.keys())

### Get the average values for each interest period

Get the actual average soil moisture for each interest period.

In [17]:
da_sm_actual_avg_for_ip = xr.concat([da_sm_binned_by_interest_period[i].mean(dim="dekad").assign_coords(dekad=i).expand_dims({"dekad": 1}) for i in interest_periods], dim="dekad")
da_sm_actual_avg_for_ip

<xarray.DataArray 'relative_soil_moisture' (dekad: 108, y: 291, x: 290)> Size: 36MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 864B 2018-01-01 ... 2020-12-21

### Get the long term average for each interest period over the years of available data


In [18]:
# Group the interest periods by year by getting the dekad number in the year for the end dekad of
# the interest period. 
grouped_by_year = toolz.groupby(lambda dekad: get_dekad_no_in_year(date=dekad), interest_periods)

Get the long term average soil moisture for each interest period.

In [19]:
long_term_avg_sm = {}
for dekad_no_in_year, interest_periods_list in grouped_by_year.items():
    long_term_avg_for_period = xr.concat([da_sm_binned_by_interest_period[i] for i in interest_periods_list], dim="dekad").mean(dim="dekad")
    long_term_avg_sm[dekad_no_in_year] = long_term_avg_for_period
    
da_sm_long_term_avg_for_ip = xr.concat([long_term_avg_sm[get_dekad_no_in_year(i)].assign_coords(dekad=i).expand_dims({"dekad": 1}) for  i in interest_periods], dim="dekad")

assert all(da_sm_actual_avg_for_ip.dekad.values == da_sm_long_term_avg_for_ip.dekad.values)

da_sm_long_term_avg_for_ip

<xarray.DataArray 'relative_soil_moisture' (dekad: 108, y: 291, x: 290)> Size: 36MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 864B 2018-01-01 ... 2020-12-21

### Get the actual length of continous deficit in each interest period

For soil moisture the run length is the number of successive dekads in an interest period below the long term average for the same interest period.

In [20]:
%%time
# For each interest period get the maximum number of successive dekads below long term average soil moisture in the interest period.
sm_run_length = []
for interest_period in interest_periods:
    # Get the dekads in the interest period
    ds = da_sm_binned_by_interest_period[interest_period]
    
    # Identify pixels which are empty for all dekads
    no_data_mask = xr.apply_ufunc(
        get_no_data_mask,
        ds,
        input_core_dims=[["dekad"]],
        vectorize=True,
        dask="allowed",
    )
    
    # Get the long term average for the interest period
    long_term_avg = da_sm_long_term_avg_for_ip.sel(dekad=interest_period)
    
    # Get the pixels where the soil moisture is below the long term average soil moisture
    mask = xr.where(ds < long_term_avg, 1, 0)
    
    # Get the maximum number of successive dekads below long term average soil moisture.
    actual_run_length = xr.apply_ufunc(
        max_consecutive_ones,
        mask,
        input_core_dims=[["dekad"]],
        vectorize=True,
        dask="allowed",
    )
    
    # Modify the run length
    modified_run_length = (actual_run_length.max() + 1) - actual_run_length
    modified_run_length = modified_run_length.where(~no_data_mask)
    sm_run_length.append(modified_run_length.assign_coords(dekad=interest_period).expand_dims({"dekad": 1}))

da_sm_actual_run_length = xr.concat(sm_run_length, dim="dekad")
da_sm_actual_run_length

CPU times: user 1min 36s, sys: 0 ns, total: 1min 36s
Wall time: 1min 35s


<xarray.DataArray 'relative_soil_moisture' (dekad: 108, y: 291, x: 290)> Size: 73MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    spatial_ref  int32 4B 6933
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
  * dekad        (dekad) datetime64[ns] 864B 2018-01-01 ... 2020-12-21

### Get the long term average of continous deficit in each interest period

In [21]:
# Get the long term average soil moisture run length for each interest period 
long_term_avg_run_lenth_sm = {}
for dekad_no_in_year, interest_periods_list in grouped_by_year.items():
    long_term_avg_for_period = da_sm_actual_run_length.sel(dekad=interest_periods_list).mean(dim="dekad")
    long_term_avg_run_lenth_sm[dekad_no_in_year] = long_term_avg_for_period
    
da_sm_long_term_avg_run_length_for_ip = xr.concat([long_term_avg_run_lenth_sm[get_dekad_no_in_year(i)].assign_coords(dekad=i).expand_dims({"dekad": 1}) for  i in interest_periods], dim="dekad")
da_sm_long_term_avg_run_length_for_ip

<xarray.DataArray 'relative_soil_moisture' (dekad: 108, y: 291, x: 290)> Size: 73MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    spatial_ref  int32 4B 6933
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
  * dekad        (dekad) datetime64[ns] 864B 2018-01-01 ... 2020-12-21

### Calculate the Soil Moisture Drought Index

In [22]:
SMDI = (da_sm_actual_avg_for_ip / da_sm_long_term_avg_for_ip) * np.sqrt((da_sm_actual_run_length / da_sm_long_term_avg_run_length_for_ip))
SMDI

<xarray.DataArray 'relative_soil_moisture' (dekad: 108, y: 291, x: 290)> Size: 73MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 864B 2018-01-01 ... 2020-12-21

In [23]:
#Scaled the Soil Moisture Drought Index
SMDI_scaled = (SMDI - SMDI.min()) / (SMDI.max() - SMDI.min())
SMDI_scaled

<xarray.DataArray 'relative_soil_moisture' (dekad: 108, y: 291, x: 290)> Size: 73MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 864B 2018-01-01 ... 2020-12-21

In [24]:
# Convert to DataFrame.
df = SMDI_scaled.to_dataframe().drop(columns="spatial_ref")
df

relative_soil_moisture
dekad      y         x                                
2018-01-01 1882500.0 3182500.0                     NaN
                     3187500.0                     NaN
                     3192500.0                     NaN
                     3197500.0                     NaN
                     3202500.0                     NaN
...                                                ...
2020-12-21 432500.0  4607500.0                     NaN
                     4612500.0                     NaN
                     4617500.0                     NaN
                     4622500.0                     NaN
                     4627500.0                     NaN

[9114120 rows x 1 columns]

In [25]:
# Convert the DataFrame to an Arrow table
table = Table.from_pandas(df)

# Get the tables existing metadata
existing_meta = table.schema.metadata

# Dump the crs of the DataArray as new metadata to JSON.
meta_json = json.dumps(dict(crs=str(SMDI_scaled.rio.crs)))

# Combine the metadata
combined_meta = {
b"xarray.metadata": meta_json.encode(),
**existing_meta,
}

# Replace the metadata.
table = table.replace_schema_metadata(combined_meta)

### Export the results in a Zip file

In [26]:
# Write the table
write_table(table, output_path, compression="GZIP")

---

## Additional information

<b> License </b> The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0).

Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

<b> Contact </b> If you need assistance, please post a question on the [DE Africa Slack channel](https://digitalearthafrica.slack.com/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).

If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

<b> Compatible datacube version :</b>

In [27]:
print(datacube.__version__)

1.8.20


**Last Tested:**

In [28]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2025-03-14'